In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from datetime import datetime, timedelta
from dateutil import relativedelta
import calendar
import numpy as np

In [2]:
dict_data = dict(json.loads(open('./data/data.json', 'r').read()))
df = pd.read_csv('./data/retrieved_data.csv', index_col='Date')

In [3]:
df.index = [str(datetime.strptime(d, '%Y-%m-%d') + timedelta(1)).split()[0] for d in df.index]
df.index = pd.to_datetime(df.index)

In [4]:
df.head()

,PPSpotAvgPrice,OilAvgPrice,PPImport,PPExport,ExchangeRate,PPFuture,OilFuture
2008-11-17,NaN,NaN,NaN,NaN,6.8289,NaN,NaN
2008-11-24,NaN,NaN,NaN,NaN,6.8317,NaN,NaN
2008-12-01,NaN,NaN,NaN,NaN,6.8349,NaN,NaN
2008-12-08,NaN,NaN,NaN,NaN,6.8482,NaN,NaN
2008-12-15,NaN,NaN,NaN,NaN,6.8451,NaN,NaN


In [5]:
raw_prices = dict_data['prices']
for row in raw_prices:
    str_date = str(datetime.fromtimestamp(row['time'])).split()[0]
    if str_date in df.index:
        df.at[str_date, 'OilFuture'] = row['price']
df = df[(df.index >= datetime(2013, 11, 17))]

In [6]:
df.head()

,PPSpotAvgPrice,OilAvgPrice,PPImport,PPExport,ExchangeRate,PPFuture,OilFuture
2013-11-18,NaN,NaN,NaN,NaN,6.1351,NaN,101.94
2013-11-25,NaN,NaN,NaN,NaN,6.1380,NaN,102.14
2013-12-02,NaN,NaN,NaN,NaN,6.1325,NaN,103.53
2013-12-09,NaN,NaN,NaN,NaN,6.1232,NaN,104.01
2013-12-16,NaN,NaN,NaN,NaN,6.1148,NaN,101.57


In [7]:
def get_last_week(month):
    first_day = datetime.strptime(str(month), '%Y%m')
    days_in_month = calendar.monthrange(first_day.year, first_day.month)[1]
    return (first_day + relativedelta.relativedelta(day=days_in_month, 
                                            weekday=relativedelta.MO(-1))).date()

In [8]:
def fill_i_e(ie_df):
    import_df = ie_df[ie_df['Trade Flow'] == 'Exports']
    export_df = ie_df[ie_df['Trade Flow'] == 'Imports']
    for month, group in import_df.groupby('Period'):
        last_week = get_last_week(month)
        if last_week.year == 2013 and last_week.month < 11:
            continue
        df.at[last_week, 'PPImport'] = "%.2f" % np.sum(group['Trade Value (US$)'])
    
    for month, group in export_df.groupby('Period'):
        last_week = get_last_week(month)
        if last_week.year == 2013 and last_week.month < 11:
            continue
        df.at[last_week, 'PPExport'] = "%.2f" % np.sum(group['Trade Value (US$)'])
    return df

In [9]:
filenames = "comtrade-{0}.csv comtrade-{1}.csv comtrade-{2}.csv \
            comtrade-{3}.csv comtrade-{4}.csv comtrade-{5}.csv ".format(3, 9, 10, 11, 12, 13).split()

for filename in filenames:
    df = fill_i_e(pd.read_csv('./data/' + filename))

In [10]:
df.head(20)

,PPSpotAvgPrice,OilAvgPrice,PPImport,PPExport,ExchangeRate,PPFuture,OilFuture
2013-11-18,NaN,NaN,NaN,NaN,6.1351,NaN,101.94
2013-11-25,NaN,NaN,375013373.00,24344624.00,6.1380,NaN,102.14
2013-12-02,NaN,NaN,NaN,NaN,6.1325,NaN,103.53
2013-12-09,NaN,NaN,NaN,NaN,6.1232,NaN,104.01
2013-12-16,NaN,NaN,NaN,NaN,6.1148,NaN,101.57
2013-12-23,NaN,NaN,NaN,NaN,6.1196,NaN,104.63
2013-12-30,NaN,NaN,421780057.00,23012790.00,6.1050,NaN,105.14
2014-01-06,NaN,NaN,NaN,NaN,6.1039,NaN,102.77
2014-01-13,NaN,NaN,NaN,NaN,6.1008,NaN,103.76
2014-01-20,NaN,NaN,NaN,NaN,6.1041,NaN,103.8


In [11]:
df.tail(20)

,PPSpotAvgPrice,OilAvgPrice,PPImport,PPExport,ExchangeRate,PPFuture,OilFuture
2018-06-25,NaN,NaN,36159357.00,7555756.00,6.4804,NaN,69.96
2018-07-02,NaN,NaN,NaN,NaN,6.6166,NaN,71.75
2018-07-09,NaN,NaN,NaN,NaN,6.6336,NaN,69.29
2018-07-16,NaN,NaN,NaN,NaN,6.6727,NaN,66.55
2018-07-23,NaN,NaN,NaN,NaN,6.7671,NaN,65.09
2018-07-30,NaN,NaN,26848232.00,9153930.00,6.7942,NaN,66.39
2018-08-06,NaN,NaN,NaN,NaN,6.8322,NaN,65.31
2018-08-13,NaN,NaN,NaN,NaN,6.8395,NaN,65.33
2018-08-20,NaN,NaN,NaN,NaN,6.8894,NaN,63.81
2018-08-27,NaN,NaN,8791063.00,720416.00,6.8710,NaN,66.50


In [ ]:
df.to_csv('./data/retrieved_data.csv')